In [1]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

# === CONFIG ===
MODEL_PATH = r"mobilenetv2_skin_type_model.keras"
TARGET_SIZE = (224, 224)
FALLBACK_CLASS = 'NOT DETECTED RESCAN YOUR FACE'
THRESHOLD = 0.5  # Fallback threshold for low confidence

# === LOAD MODEL ===
model = load_model(MODEL_PATH)

# === CLASS LABELS ===
labels = {0: 'Combination', 1: 'Dry', 2: 'Normal', 3: 'Oily', 4: 'Sensitive'}

# === MEDIAPIPE FACE DETECTOR ===
mp_face_detection = mp.solutions.face_detection

def preprocess_face(face_img):
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    face_img = cv2.resize(face_img, TARGET_SIZE)
    face_array = img_to_array(face_img) / 255.0  # Normalize
    return np.expand_dims(face_array, axis=0)

def give_feedback_and_predict(face):
    input_data = preprocess_face(face)
    preds = model.predict(input_data, verbose=0)[0]

    print("\n🔍 Softmax Probabilities:")
    for i, prob in enumerate(preds):
        print(f"{labels[i]}: {prob:.4f}")

    top_idx = np.argmax(preds)
    confidence = preds[top_idx]

    if confidence < THRESHOLD:
        final_label = FALLBACK_CLASS
    else:
        final_label = labels[top_idx]

    print(f"\n✅ Predicted Skin Type: {final_label} (Confidence: {confidence:.2f})")
    return final_label, preds

def is_inside_oval(face_box, frame_shape, oval_center, oval_axes):
    x, y, w, h = face_box
    cx = x + w // 2
    cy = y + h // 2
    dx = (cx - oval_center[0]) / oval_axes[0]
    dy = (cy - oval_center[1]) / oval_axes[1]
    return (dx**2 + dy**2) <= 1.0  # Inside oval equation

def check_lighting(face_img):
    gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray)
    return brightness > 80  # Threshold for brightness

# === OPENCV CAPTURE LOOP ===
cap = cv2.VideoCapture(0)

with mp_face_detection.FaceDetection(min_detection_confidence=0.6) as face_detection:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame.")
            break

        h, w, _ = frame.shape
        oval_center = (w // 2, h // 2)
        oval_axes = (150, 180)  # Width, Height of oval

        # Draw oval on frame
        cv2.ellipse(frame, oval_center, oval_axes, 0, 0, 360, (0, 255, 0), 2)
        cv2.putText(frame, "Align your face inside the oval & press 'c' to capture", 
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Detect face
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(image_rgb)

        feedback_msg = ""
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                x = int(bbox.xmin * w)
                y = int(bbox.ymin * h)
                width = int(bbox.width * w)
                height = int(bbox.height * h)

                face_crop = frame[y:y+height, x:x+width]

                # Alignment check
                if not is_inside_oval((x, y, width, height), frame.shape, oval_center, oval_axes):
                    feedback_msg = "Align your face inside the oval"
                elif not check_lighting(face_crop):
                    feedback_msg = "Increase lighting"
                else:
                    feedback_msg = "Good alignment - Press 'c' to capture"

                cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

                # Capture and predict
                key = cv2.waitKey(1) & 0xFF
                if key == ord('c') and feedback_msg.startswith("Good"):
                    label, probs = give_feedback_and_predict(face_crop)
                    cv2.putText(frame, f"Predicted: {label}", (10, h - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

        else:
            feedback_msg = "No Face Detected"

        # Show feedback on frame
        cv2.putText(frame, feedback_msg, (10, h - 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255) if "No" in feedback_msg or "Align" in feedback_msg or "lighting" in feedback_msg else (0, 255, 0), 2)

        cv2.imshow("Skin Type Detection", frame)

        if cv2.waitKey(5) & 0xFF == 27:  # ESC to exit
            break

cap.release()
cv2.destroyAllWindows()


c:\Users\anush\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))



🔍 Softmax Probabilities:
Combination: 0.4484
Dry: 0.0036
Normal: 0.1012
Oily: 0.0920
Sensitive: 0.3548

✅ Predicted Skin Type: NOT DETECTED RESCAN YOUR FACE (Confidence: 0.45)

🔍 Softmax Probabilities:
Combination: 0.3971
Dry: 0.0228
Normal: 0.0444
Oily: 0.1072
Sensitive: 0.4285

✅ Predicted Skin Type: NOT DETECTED RESCAN YOUR FACE (Confidence: 0.43)

🔍 Softmax Probabilities:
Combination: 0.3021
Dry: 0.0082
Normal: 0.0123
Oily: 0.0538
Sensitive: 0.6236

✅ Predicted Skin Type: Sensitive (Confidence: 0.62)

🔍 Softmax Probabilities:
Combination: 0.3200
Dry: 0.0072
Normal: 0.0095
Oily: 0.0609
Sensitive: 0.6024

✅ Predicted Skin Type: Sensitive (Confidence: 0.60)


In [13]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tkinter import Tk, filedialog
import mediapipe as mp

# === CONFIG ===
MODEL_PATH = r"mobilenetv2_skin_type_model.keras"
TARGET_SIZE = (224, 224)
FALLBACK_CLASS = 'Normal'
THRESHOLD = 0.2  # Only use fallback if ALL class scores are below this

# === LOAD MODEL ===
model = load_model(MODEL_PATH)

# === CLASS LABELS ===
labels = {0: 'Combination', 1: 'Dry', 2: 'Normal', 3: 'Oily', 4: 'Sensitive'}

# === MEDIAPIPE FACE DETECTOR ===
mp_face_detection = mp.solutions.face_detection

def preprocess_face(face_img):
    """Resize and normalize face image for model input."""
    face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)
    face_img = cv2.resize(face_img, TARGET_SIZE)
    face_array = img_to_array(face_img) / 255.0
    return np.expand_dims(face_array, axis=0)

def give_prediction(face_img):
    """Predict skin type from face image with improved fallback logic."""
    input_data = preprocess_face(face_img)
    preds = model.predict(input_data, verbose=0)[0]

    print("\n🔍 Softmax Probabilities:")
    for i, prob in enumerate(preds):
        print(f"{labels[i]}: {prob:.4f}")

    top_idx = np.argmax(preds)
    max_prob = preds[top_idx]

    # Fallback only if ALL probabilities are below threshold
    if max_prob < THRESHOLD:
        final_label = FALLBACK_CLASS
    else:
        final_label = labels[top_idx]

    print(f"\n✅ Predicted Skin Type: {final_label} (Confidence: {max_prob:.2f})")
    return final_label

def predict_from_uploaded_image():
    """Open file dialog, detect face, predict skin type, and show labeled image."""
    Tk().withdraw()  # Hide main tkinter window
    file_path = filedialog.askopenfilename(
        title="Select an Image",
        filetypes=[("Image files", "*.jpg *.jpeg *.png")]
    )
    if not file_path:
        print("No file selected.")
        return

    image = cv2.imread(file_path)
    if image is None:
        print("Error loading image.")
        return

    # Detect face with Mediapipe
    with mp_face_detection.FaceDetection(min_detection_confidence=0.6) as face_detection:
        results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        if not results.detections:
            print("❌ No face detected in the image.")
            return

        for detection in results.detections:
            bbox = detection.location_data.relative_bounding_box
            h, w, _ = image.shape
            x = int(bbox.xmin * w)
            y = int(bbox.ymin * h)
            width = int(bbox.width * w)
            height = int(bbox.height * h)

            # Crop face
            face_crop = image[y:y+height, x:x+width]
            if face_crop.size == 0:
                print("❌ Face region is empty.")
                return

            # Predict skin type
            label = give_prediction(face_crop)

            # Draw box and label
            cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
            cv2.putText(image, f"Skin Type: {label}", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 255), 2)

    # Show the result
    cv2.imshow("Skin Type Prediction", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# Run standalone
if __name__ == "__main__":
    predict_from_uploaded_image()


No file selected.


In [2]:
import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import os

# === CONFIG ===
MODEL_PATH = r"mobilenetv2_skin_type_model.keras"
TARGET_SIZE = (224, 224)
FALLBACK_CLASS = 'Normal'
THRESHOLD = 0.5  # Fallback threshold for low confidence

# === LOAD MODEL ===
try:
    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Model file not found at: {MODEL_PATH}")
    model = load_model(MODEL_PATH)
    print("✅ Model loaded successfully.")
except (FileNotFoundError, Exception) as e:
    print(f"❌ Error loading the model: {e}")
    # Exit the program if the model cannot be loaded
    exit()

# === CLASS LABELS ===
labels = {0: 'Combination', 1: 'Dry', 2: 'Normal', 3: 'Oily', 4: 'Sensitive'}

# === MEDIAPIPE FACE DETECTOR ===
mp_face_detection = mp.solutions.face_detection

def preprocess_face(face_img):
    """Preprocesses a face image for model prediction."""
    try:
        face_img = cv2.cvtColor(face_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
        face_img = cv2.resize(face_img, TARGET_SIZE)
        face_array = img_to_array(face_img) / 255.0  # Normalize
        return np.expand_dims(face_array, axis=0)
    except Exception as e:
        print(f"❌ Error during face preprocessing: {e}")
        return None

def give_feedback_and_predict(face):
    """Makes a prediction on a cropped face and provides feedback."""
    input_data = preprocess_face(face)
    if input_data is None:
        return None, None
    
    try:
        preds = model.predict(input_data, verbose=0)[0]
    except Exception as e:
        print(f"❌ Error during model prediction: {e}")
        return None, None

    print("\n🔍 Softmax Probabilities:")
    for i, prob in enumerate(preds):
        print(f"{labels[i]}: {prob:.4f}")

    top_idx = np.argmax(preds)
    confidence = preds[top_idx]

    if confidence < THRESHOLD:
        final_label = FALLBACK_CLASS
        print(f"Low confidence ({confidence:.2f} < {THRESHOLD:.2f}). Falling back to '{FALLBACK_CLASS}'.")
    else:
        final_label = labels[top_idx]

    print(f"\n✅ Predicted Skin Type: {final_label} (Confidence: {confidence:.2f})")
    return final_label, preds

def is_inside_oval(face_box, oval_center, oval_axes):
    """Checks if the face bounding box is inside the oval guide."""
    x, y, w, h = face_box
    cx = x + w // 2
    cy = y + h // 2
    dx = (cx - oval_center[0]) / oval_axes[0]
    dy = (cy - oval_center[1]) / oval_axes[1]
    return (dx**2 + dy**2) <= 1.0  # Inside oval equation

def check_lighting(face_img):
    """Checks for sufficient lighting by calculating average brightness."""
    try:
        gray = cv2.cvtColor(face_img, cv2.COLOR_BGR2GRAY)
        brightness = np.mean(gray)
        return brightness > 80  # Threshold for brightness
    except Exception as e:
        print(f"❌ Error checking lighting: {e}")
        return False

# === OPENCV CAPTURE LOOP ===
cap = None  # Initialize cap to None
try:
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    with mp_face_detection.FaceDetection(min_detection_confidence=0.6) as face_detection:
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame. Exiting...")
                break

            h, w, _ = frame.shape
            oval_center = (w // 2, h // 2)
            oval_axes = (150, 180)  # Width, Height of oval

            # Draw oval on frame
            cv2.ellipse(frame, oval_center, oval_axes, 0, 0, 360, (0, 255, 0), 2)
            cv2.putText(frame, "Align your face inside the oval & press 'c' to capture",
                        (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

            # Detect face
            image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = face_detection.process(image_rgb)

            feedback_msg = ""
            prediction_text = ""
            
            if results.detections:
                for detection in results.detections:
                    bbox = detection.location_data.relative_bounding_box
                    x = int(bbox.xmin * w)
                    y = int(bbox.ymin * h)
                    width = int(bbox.width * w)
                    height = int(bbox.height * h)

                    face_crop = frame[y:y+height, x:x+width]

                    # Alignment check
                    if not is_inside_oval((x, y, width, height), oval_center, oval_axes):
                        feedback_msg = "Align your face inside the oval"
                    elif not check_lighting(face_crop):
                        feedback_msg = "Increase lighting"
                    else:
                        feedback_msg = "Good alignment - Press 'c' to capture"
                    
                    # Draw rectangle around the face
                    cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)

                    # Capture and predict
                    key = cv2.waitKey(1) & 0xFF
                    if key == ord('c') and feedback_msg.startswith("Good"):
                        label, probs = give_feedback_and_predict(face_crop)
                        if label:
                            prediction_text = f"Predicted: {label}"
            else:
                feedback_msg = "No Face Detected"

            # Show feedback and prediction text on frame
            cv2.putText(frame, feedback_msg, (10, h - 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255) if "No" in feedback_msg or "Align" in feedback_msg or "lighting" in feedback_msg else (0, 255, 0), 2)
            
            if prediction_text:
                cv2.putText(frame, prediction_text, (10, h - 20),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 255), 2)

            cv2.imshow("Skin Type Detection", frame)

            if cv2.waitKey(5) & 0xFF == 27:  # ESC to exit
                break

except (IOError, Exception) as e:
    print(f"❌ An error occurred: {e}")
finally:
    # This block will always execute, whether an exception occurred or not
    if cap is not None:
        cap.release()
    cv2.destroyAllWindows()
    print("Cleanup complete. Application closed.")

✅ Model loaded successfully.

🔍 Softmax Probabilities:
Combination: 0.0541
Dry: 0.0002
Normal: 0.0036
Oily: 0.0118
Sensitive: 0.9304

✅ Predicted Skin Type: Sensitive (Confidence: 0.93)
Cleanup complete. Application closed.
